# Processing RNA-seq data

Trying some alternative pipelines for processing RNA-seq data using existing tools: sleuth + DESeq. Will collect data to generate a summary of the changes (similarities and differences) between differential expression results that come out of these tools, dependent on the pipeline and tool's analysis methods used.

All are starting out with the same RNA-seq results mapped to ensembl transcriptome using kallisto. 

List of existing models/pipelines:
- [x] One sleuth object constructed and trained with a full model, supplied gene-to-transcript mapping, individual wald-tests based on condition to pull out DE transcripts based on these results 
    * notes from specs on expected behavior:
- [ ] Pairwise sleuth objects generated per condition, with a wald test to analyze the DE transcripts for that condition
- [ ] DESeq2 pipeline with only the two variables ('CTCF_AID_untreated' vs some auxin treatment) 

## Creating sleuth full model + Wald Test using only two conditions

In [2]:
library("sleuth")
library(stringr)
library(dplyr)

suppressMessages({
  library("sleuth")
})

Warning message:
"package 'stringr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [3]:
# Getting location of the kalliso outputs
elphege_proj_files = "/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652"
sample_id <- dir(file.path(elphege_proj_files,"SRP106652"))
kal_dirs <- file.path(elphege_proj_files, "SRP106652", sample_id, "kallisto-mm9")

files <- file.path(kal_dirs, "abundance.h5")
names(files) <- str_split_fixed(files,'/',9)[,7]

In [4]:
# Getting "Run" and "condition" from Experiment Name of the SRA deposits
sra_summary <- read.table(
                    file.path(elphege_proj_files, 
                             "metadata", "sra_result_summary.csv"), 
                              header = TRUE, sep = ",", stringsAsFactors=FALSE)

# # Parse experiment name to get 'condition' -- str_split_fixed(string, pattern, n)
# # also Removing replicate number so that sleuth can do some grouping

conditions = data.frame(
                condition=str_split_fixed( str_split_fixed(
                             sra_summary$Experiment.Title,
                             "seq_", 4)[,2] ,'_rep',2)[,1])

sra_summary <- cbind(sra_summary,  conditions)
    
    
run_info <- read.table(file.path(elphege_proj_files, "metadata", "SraRunInfo.csv"), header = TRUE, sep = ",", stringsAsFactors=FALSE)

# # merge sra_result file and sraRunInfo to get the condition
# # (buried in 'Experiment Name' in sra_result_summary -- put in 'condition')
sra_info_combined <- merge(run_info, sra_summary, 
                             by.x=c("Experiment"),
                             by.y=c("Experiment.Accession"),
                             all.x=TRUE)

s2c <- dplyr::select(sra_info_combined, Experiment, Run, condition, Library.Strategy, Library.Selection)
s2c <- dplyr::select(s2c, sample = Run, condition)
s2c$condition <- gsub('-', '_', s2c$condition) # apparently R wants  of factors in the design to only have letters, numbers, '_' and '.'
s2c$condition <- as.factor(s2c$condition)        # Convert character column to factor (not necessary, just suppresses warning below)

# Adding kallisto result directory file path so sleuth knows where to find the files for each sample
s2c <- dplyr::mutate(s2c, path = kal_dirs)

# Reformatting condition so that when converted to levels, we can put the base case first
## note: default behavior for sleuth will order the levels alphabetically and perform comparisons based on this order
s2c$condition = as.factor(s2c$condition)

levels(s2c$condition)

# Getting rid of whitespace didn't work? so just going to try to se the base level
s2c$condition <- relevel(s2c$condition, ref = "CTCF_AID_untreated")

[1] "CTCF_AID_auxin1day"     "CTCF_AID_auxin2days"    "CTCF_AID_auxin4days"   
[4] "CTCF_AID_untreated"     "CTCF_AID_washoff2days"  "WT_untagged_auxin2days"
[7] "WT_untagged_auxin4days" "WT_untagged_untreated"

In [5]:
# Creating a sample2condition set for only the untreated vs. variable of interest
condition = 'CTCF_AID_auxin1day'

s2c_untreated_vs_auxin1day = s2c[ (s2c$condition=='CTCF_AID_untreated') | (s2c$condition==condition), ]

s2c_untreated_vs_auxin1day

,sample,condition,path
,<chr>,<fct>,<chr>
1,SRR5517500,CTCF_AID_untreated,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517500/kallisto-mm9
2,SRR5517501,CTCF_AID_auxin1day,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517501/kallisto-mm9
6,SRR5517505,CTCF_AID_untreated,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517505/kallisto-mm9
7,SRR5517506,CTCF_AID_auxin1day,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517506/kallisto-mm9
11,SRR5517510,CTCF_AID_untreated,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517510/kallisto-mm9
12,SRR5517511,CTCF_AID_auxin1day,/project/fudenber_735/GEO/nora_2017_rna-seq_SRP106652/SRP106652/SRR5517511/kallisto-mm9


In [51]:
so_1day <- sleuth_prep(s2c_untreated_vs_auxin1day, extra_bootstrap_summary = TRUE)

reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

48729 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps





In [52]:
so_1day <- sleuth_fit(so_1day, ~condition, 'full')

fitting measurement error models

shrinkage estimation

computing variance of betas



In [34]:
#so_1day <- sleuth_fit(so_1day, ~1, 'reduced')

fitting measurement error models

shrinkage estimation

computing variance of betas



In [53]:
models(so_1day)

[  full  ]
formula:  ~condition 
data modeled:  obs_counts 
transform sync'ed:  TRUE 
coefficients:
	(Intercept)
 	conditionCTCF_AID_auxin1day


In [54]:
# Running wald test to do pair-wise comparison of these two guys
so_1day <- sleuth_wt(so_1day, which_beta = paste("condition",condition, sep=""))
wt_1day_res <- sleuth_results(so_1day, test = paste("condition", condition, sep=""), test_type='wt', show_all=TRUE)

In [55]:
print(dim(wt_1day_res))
print(toString(count(wt_1day_res)))
head(wt_1day_res)

[1] 88366    11
[1] "88366"


,target_id,pval,qval,b,se_b,mean_obs,var_obs,tech_var,sigma_sq,smooth_sigma_sq,final_sigma_sq
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSMUST00000168900,1.492824e-21,7.274380e-17,-1.913805,0.2007049,6.664804,1.1094981,0.0034162545,0.0099630455,0.05700740,0.05700740
2,ENSMUST00000102763,7.946625e-21,1.936155e-16,-1.671124,0.1785320,7.230924,0.8743488,0.0011131293,0.0445774204,0.04669739,0.04669739
3,ENSMUST00000006451,5.340841e-20,8.675129e-16,-1.848426,0.2018615,6.753079,1.0332171,0.0059649274,0.0043014988,0.05515719,0.05515719
4,ENSMUST00000029644,5.457137e-19,6.648021e-15,-1.830264,0.2055884,6.536837,1.0083426,0.0034754271,0.0007535977,0.05992444,0.05992444
5,ENSMUST00000034097,4.881522e-15,4.757433e-11,-1.247161,0.1592813,8.471965,0.4970679,0.0003769877,0.0376788179,0.03536896,0.03767882
6,ENSMUST00000032839,1.443162e-14,1.172064e-10,-1.980281,0.2574310,5.620549,1.2257721,0.0062543955,0.0553932438,0.09315165,0.09315165


In [56]:
sig_res <- dplyr::filter(wt_1day_res, qval <= 0.05)
sig_up <- dplyr::filter(sig_res, b <= 0)
sig_down <- dplyr::filter(sig_res, b > 0)

In [66]:
print(paste("Number of significant DE genes for condition:", condition, toString(count(sig_res)), sep=" "))
print(paste("beta < 0 [up]:", toString(count(sig_up)), sep=" "))
print(paste("beta > 0 [down]:", toString(count(sig_down)), sep=" "))
print('')

[1] "Number of significant DE genes for condition: CTCF_AID_auxin1day 103"
[1] "beta < 0 [up]: 80"
[1] "beta > 0 [down]: 23"
[1] ""


## Pairwise WT for all

In [6]:
# Training and getting details of pairwise model for each of the conditions
for (cond in levels(s2c$condition)) {
    
    if (cond != "CTCF_AID_untreated") {
        
        # build sleuth object and model for this condition
        s2c_untreated_vs_treatment = s2c[ (s2c$condition=='CTCF_AID_untreated') | (s2c$condition==cond), ]
        so <- sleuth_prep(s2c_untreated_vs_treatment, extra_bootstrap_summary = TRUE)
        so <- sleuth_fit(so, ~condition, 'full')
        
        # perform wald test and extract results
        so <- sleuth_wt(so, which_beta = paste("condition",cond, sep=""))
        wt_res <- sleuth_results(so, test = paste("condition", cond, sep=""), test_type='wt', show_all=TRUE)
        
        # Save full results from this as .csv 
        write.csv(wt_res,file.path(elphege_proj_files, "WT-mm9-results", "pairwise-sleuth-wt", paste("pairwite_sleuth_wt_", cond, ".csv", sep="")))
        
        sig_res <- dplyr::filter(wt_res, qval <= 0.05)
        sig_up <- dplyr::filter(sig_res, b <= 0)
        sig_down <- dplyr::filter(sig_res, b > 0)
        
        print("=== Some basic stats ===")
        print(paste("Number of significant DE genes for condition:", cond, toString(count(sig_res)), sep=" "))
        print(paste("beta < 0 [up]:", toString(count(sig_up)), sep=" "))
        print(paste("beta > 0 [down]:", toString(count(sig_down)), sep=" "))
        print('')
        
        }
    }

reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

48729 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps



fitting measurement error models

shrinkage estimation

computing variance of betas



[1] "=== Some basic stats ==="
[1] "Number of significant DE genes for condition: CTCF_AID_auxin1day 103"
[1] "beta < 0 [up]: 80"
[1] "beta > 0 [down]: 23"
[1] ""


reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

49577 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps



fitting measurement error models

shrinkage estimation

7 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surface to extrapolate the missing values.
These are the target ids with NA values: ENSMUST00000024843, ENSMUST00000067075, ENSMUST00000101065, ENSMUST00000115561, ENSMUST00000143407, ENSMUST00000149144, ENSMUST00000152167

computing variance of betas



[1] "=== Some basic stats ==="
[1] "Number of significant DE genes for condition: CTCF_AID_auxin2days 269"
[1] "beta < 0 [up]: 118"
[1] "beta > 0 [down]: 151"
[1] ""


reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

50352 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps



fitting measurement error models

shrinkage estimation

8 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surface to extrapolate the missing values.
These are the target ids with NA values: ENSMUST00000115335, ENSMUST00000119141, ENSMUST00000130376, ENSMUST00000136744, ENSMUST00000146174, ENSMUST00000147633, ENSMUST00000150543, ENSMUST00000172772

computing variance of betas



...

[1] "=== Some basic stats ==="
[1] "Number of significant DE genes for condition: WT_untagged_auxin2days 10"
[1] "beta < 0 [up]: 4"
[1] "beta > 0 [down]: 6"
[1] ""


reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

49725 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps



fitting measurement error models

shrinkage estimation

computing variance of betas



[1] "=== Some basic stats ==="
[1] "Number of significant DE genes for condition: WT_untagged_auxin4days 17"
[1] "beta < 0 [up]: 2"
[1] "beta > 0 [down]: 15"
[1] ""


reading in kallisto results

dropping unused factor levels

.
.
.
.
.
.


normalizing est_counts

50357 targets passed the filter

normalizing tpm

merging in metadata

summarizing bootstraps



fitting measurement error models

shrinkage estimation

1 NA values were found during variance shrinkage estimation due to mean observation values outside of the range used for the LOESS fit.
The LOESS fit will be repeated using exact computation of the fitted surface to extrapolate the missing values.
These are the target ids with NA values: ENSMUST00000118207

computing variance of betas



[1] "=== Some basic stats ==="
[1] "Number of significant DE genes for condition: WT_untagged_untreated 21"
[1] "beta < 0 [up]: 10"
[1] "beta > 0 [down]: 11"
[1] ""
